### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-08-25


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-08-25
Yesterday: 2025-08-22


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-08-22'
ORDER BY a.name



(16, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-22' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-22,2025-08-22,9.55,9.55,41400,9.60,9.55,B,SET,0.00,0.00
1,CPF,2025-08-22,2025-08-22,24.30,24.30,16229198,24.50,24.00,B,SET50,0.00,0.00
2,GVREIT,2025-08-21,2025-08-22,6.35,6.40,1097507,6.40,6.25,B,SET,0.05,0.79
3,JMART,2025-08-22,2025-08-22,8.35,8.35,7840481,8.55,8.35,B,SET50,0.00,0.00
4,JMT,2025-08-21,2025-08-22,11.40,11.40,20103870,11.70,11.20,B,SET50,0.00,0.00
5,ORI,2025-08-21,2025-08-22,2.26,2.26,19851426,2.36,2.20,B,SET100,0.00,0.00
6,PTT,2025-08-20,2025-08-22,32.00,32.00,154139483,32.25,31.75,B,SET50,0.00,0.00
7,RCL,2025-08-22,2025-08-22,26.75,26.75,2122258,27.25,26.75,B,SET100,0.00,0.00
8,TFFIF,2025-08-21,2025-08-22,6.05,6.05,3035522,6.10,6.00,B,SET,0.00,0.00
9,WHAIR,2025-08-22,2025-08-22,5.40,5.40,283902,5.40,5.35,B,SET,0.00,0.00


In [20]:
df.shape

(16, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AIMIRT,2025-08-22,2025-08-22,9.55,9.55,"41,400",9.60,9.55,0.00%,B
1,CPF,2025-08-22,2025-08-22,24.30,24.30,"16,229,198",24.50,24.00,0.00%,B
2,GVREIT,2025-08-21,2025-08-22,6.35,6.40,"1,097,507",6.40,6.25,0.79%,B
3,JMART,2025-08-22,2025-08-22,8.35,8.35,"7,840,481",8.55,8.35,0.00%,B
4,JMT,2025-08-21,2025-08-22,11.40,11.40,"20,103,870",11.70,11.20,0.00%,B
5,ORI,2025-08-21,2025-08-22,2.26,2.26,"19,851,426",2.36,2.20,0.00%,B
6,PTT,2025-08-20,2025-08-22,32.00,32.00,"154,139,483",32.25,31.75,0.00%,B
7,RCL,2025-08-22,2025-08-22,26.75,26.75,"2,122,258",27.25,26.75,0.00%,B
8,TFFIF,2025-08-21,2025-08-22,6.05,6.05,"3,035,522",6.10,6.00,0.00%,B
9,WHAIR,2025-08-22,2025-08-22,5.40,5.40,"283,902",5.40,5.35,0.00%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(16, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(16, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AIMIRT,2025-08-22,2025-08-22,9.55,9.55,41400,9.60,9.55,0.00,B,9.70,0.15,180935,2025-08-25
1,CPF,2025-08-22,2025-08-22,24.30,24.30,16229198,24.50,24.00,0.00,B,24.20,-0.10,13263173,2025-08-25
2,GVREIT,2025-08-21,2025-08-22,6.35,6.40,1097507,6.40,6.25,0.79,B,6.45,0.05,282053,2025-08-25
3,JMART,2025-08-22,2025-08-22,8.35,8.35,7840481,8.55,8.35,0.00,B,8.60,0.25,21677743,2025-08-25
4,JMT,2025-08-21,2025-08-22,11.40,11.40,20103870,11.70,11.20,0.00,B,12.20,0.80,31135734,2025-08-25
5,ORI,2025-08-21,2025-08-22,2.26,2.26,19851426,2.36,2.20,0.00,B,2.28,0.02,22151597,2025-08-25
6,PTT,2025-08-20,2025-08-22,32.00,32.00,154139483,32.25,31.75,0.00,B,32.25,0.25,7007089,2025-08-25
7,RCL,2025-08-22,2025-08-22,26.75,26.75,2122258,27.25,26.75,0.00,B,27.00,0.25,1303376,2025-08-25
8,TFFIF,2025-08-21,2025-08-22,6.05,6.05,3035522,6.10,6.00,0.00,B,6.05,0.00,2378648,2025-08-25
9,WHAIR,2025-08-22,2025-08-22,5.40,5.40,283902,5.40,5.35,0.00,B,5.50,0.10,275470,2025-08-25


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-08-22 2025-08-25   9.55   9.70   1.57%      41,400   9.70   9.55 B Update
CPF      2025-08-22 2025-08-25  24.30  24.20  -0.41%  16,229,198  24.50  24.00 B Update
DIF      2025-08-21 2025-08-25   8.20   8.25   0.61%  12,794,138   8.25   8.10 S Update
GVREIT   2025-08-21 2025-08-25   6.35   6.45   1.57%   1,097,507   6.45   6.25 B Update
IVL      2025-08-22 2025-08-25  22.80  23.30   2.19%  26,354,872  23.30  22.60 S Update
JMART    2025-08-22 2025-08-25   8.35   8.60   2.99%   7,840,481   8.60   8.35 B Update
JMT      2025-08-21 2025-08-25  11.40  12.20   7.02%  20,103,870  12.20  11.20 B Update
MCS      2025-08-22 2025-08-25   8.45   8.55   1.18%     521,150   8.65   8.35 S Update
NER      2025-08-25 2025-08-25   4.30   4.34   0.93%  44,260,178   4.60   4.24 S Insert
ORI      2025-08-21 2025-08-25  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '{}' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name
""".format(today)
print(sql)


    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '2025-08-25' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(16, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-22,2025-08-25,9.55,9.70,"82,800",9.70,9.55,B,SET,0.15,1.57%
1,CPF,2025-08-22,2025-08-25,24.30,24.20,"32,458,396",24.50,24.00,B,SET50,-0.10,-0.41%
2,GVREIT,2025-08-21,2025-08-25,6.35,6.45,"2,195,014",6.45,6.25,B,SET,0.10,1.57%
3,JMART,2025-08-22,2025-08-25,8.35,8.60,"15,680,962",8.60,8.35,B,SET50,0.25,2.99%
4,JMT,2025-08-21,2025-08-25,11.40,12.20,"40,207,740",12.20,11.20,B,SET50,0.80,7.02%
5,ORI,2025-08-21,2025-08-25,2.26,2.28,"39,702,852",2.36,2.20,B,SET100,0.02,0.88%
6,PTT,2025-08-20,2025-08-25,32.00,32.25,"308,278,966",32.25,31.75,B,SET50,0.25,0.78%
7,RCL,2025-08-22,2025-08-25,26.75,27.00,"4,244,516",27.25,26.75,B,SET100,0.25,0.93%
8,TFFIF,2025-08-21,2025-08-25,6.05,6.05,"6,071,044",6.10,6.00,B,SET,0.00,0.00%
9,WHAIR,2025-08-22,2025-08-25,5.40,5.50,"567,804",5.50,5.35,B,SET,0.10,1.85%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
4,JMT,2025-08-21,2025-08-25,11.40,12.20,"40,207,740",12.20,11.20,B,SET50,0.80,7.02%


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-25 20:13:45
